# Predict on noisy tracks and save to disk/bucket

In [1]:
import utils

In [2]:
from multiprocessing import set_start_method, cpu_count
set_start_method("spawn")
num_cpus = cpu_count()
print('{} available cpus'.format(num_cpus))

4 available cpus


## 1. Load data

In [3]:
!gsutil -m cp -n -r gs://capstone_datasets/librispeech/test/predictions/* ./predictions/

Skipping existing item: file://./predictions/lr_clean_test_ds_1000Hz_w2v2_base_960h/dataset.arrow
Skipping existing item: file://./predictions/lr_clean_test_ds_1000Hz_w2v2_base_960h/dataset_info.json
Skipping existing item: file://./predictions/lr_clean_test_ds_1000Hz_w2v2_base_960h/state.json
Skipping existing item: file://./predictions/lr_clean_test_ds_16000Hz_w2v2_base_960h/dataset.arrow
Skipping existing item: file://./predictions/lr_clean_test_ds_16000Hz_w2v2_base_960h/dataset_info.json
Skipping existing item: file://./predictions/lr_clean_test_ds_2000Hz_w2v2_base_960h/dataset.arrow
Skipping existing item: file://./predictions/lr_clean_test_ds_2000Hz_w2v2_base_960h/dataset_info.json
Skipping existing item: file://./predictions/lr_clean_test_ds_16000Hz_w2v2_base_960h/state.json
Skipping existing item: file://./predictions/lr_clean_test_ds_2000Hz_w2v2_base_960h/state.json
Skipping existing item: file://./predictions/lr_clean_test_ds_4000Hz_w2v2_base_960h/dataset_info.json
Skipping e

In [4]:
dataset = utils.load_from_disk(utils.os.path.join(utils.predictions_path, 'lr_clean_test_w2v2_base_960h'))

## 2. Add noise

In [5]:
noise_percentage_factor = 0.06

In [6]:
%%time

print('adding noise...')
dataset = dataset.map(utils.map_to_noisy, fn_kwargs={'sample_rate' : 16000, 'noise_percentage_factor' : noise_percentage_factor, 'noise_type'  : 'white'}, 
    num_proc=num_cpus, writer_batch_size=300) # decrease writer_batch_size to avoid OOM issues

adding noise...
     

#0:   0%|          | 0/655 [00:00<?, ?ex/s]

#1:   0%|          | 0/655 [00:00<?, ?ex/s]

#2:   0%|          | 0/655 [00:00<?, ?ex/s]

#3:   0%|          | 0/655 [00:00<?, ?ex/s]

CPU times: user 2.05 s, sys: 564 ms, total: 2.61 s
Wall time: 2min 21s


## 3. Compute prediction

In [7]:
tokenizer, model = utils.load_wav2vec_model("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/opt/conda/lib/python3.7/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:757: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
%%time

print('computing prediction...')
dataset = dataset.map(utils.map_to_pred, fn_kwargs={"model": model, "tokenizer": tokenizer}, writer_batch_size=1000)

computing prediction...


  0%|          | 0/2620 [00:00<?, ?ex/s]

CPU times: user 5min 44s, sys: 36.9 s, total: 6min 21s
Wall time: 6min 22s


## 3. Save to disk

In [9]:
dataset.save_to_disk(utils.os.path.join(utils.predictions_path, 'lr_clean_test_ns_'+ str(int(100 * noise_percentage_factor)) + '%_w2v2_base_960h'))

## 4. Compute WER

In [10]:
wer = utils.wer(dataset["ground_truth"], dataset["transcription"])
print('wer=', round(100 * wer, 1), '%.')

wer= 92.5 %.


## 5. Send all downsampled datasets in the bucket

In [1]:
#!gsutil -m cp -n -r ./predictions/ gs://capstone_datasets/librispeech/test/

Copying file://./predictions/lr_clean_test_ds_2000Hz_w2v2_base_960h/state.json [Content-Type=application/json]...
Copying file://./predictions/lr_clean_test_ds_2000Hz_w2v2_base_960h/dataset_info.json [Content-Type=application/json]...
Copying file://./predictions/lr_clean_test_ds_500Hz_w2v2_base_960h/dataset_info.json [Content-Type=application/json]...
Copying file://./predictions/lr_clean_test_ds_2000Hz_w2v2_base_960h/dataset.arrow [Content-Type=application/octet-stream]...
Copying file://./predictions/lr_clean_test_ds_4000Hz_w2v2_base_960h/dataset_info.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-